In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_in_out_daily

(calday DATE,
  store_id STRING,
  product_id STRING,
  in_out_type STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")


In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_in_out_daily
WHERE calday = DATE_SUB(DATE(current_timestamp() + INTERVAL 7 HOURS), 1)""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_in_out_daily
WITH temp1 AS (
    SELECT 
        calday,
        store_id,
        product_id,
        LEAD(calday) OVER (PARTITION BY store_id, product_id ORDER BY calday DESC) AS lead_calday,
        LAG(calday) OVER (PARTITION BY store_id, product_id ORDER BY calday DESC) AS lag_calday
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his
    WHERE calday BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
)
SELECT 
    DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1) AS calday,
    t.store_id,
    t.product_id,
    'IN' AS in_out_type
FROM temp1 t
WHERE t.calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
  AND lead_calday IS NULL

UNION ALL

SELECT 
    t.calday,
    t.store_id,
    t.product_id,
    'OUT' 
FROM TEMP1 T
WHERE T.CALDAY = DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)
AND LAG_CALDAY IS NULL
""")
